# Data Pipeline

## 0. Load Required Libraries

In [1]:
from tqdm import tqdm
import pandas as pd
import numpy as np
import yfinance as yf
from datetime import date
import joblib
import os
import yaml
import src.util as util
from sklearn.model_selection import TimeSeriesSplit


## 1. Load Configuration File

In [2]:
def read_raw_data(config: dict) -> pd.DataFrame:
    # Load and define stock ticker list at IDX
    stock_list = pd.read_excel(config['raw_dataset_dir'])

    # Add new column with a value suitable to ticker name at yfinance
    stock_list['ticker.jk'] = stock_list['Kode'] + config['ticker_ext']

    # Take only the needed column and change it from df to list
    ticker_list = stock_list['ticker.jk'].tolist()
    
    # Define the date range parameter
    start_date = config['start_date']
    end_date = date.today()
    interval = config['interval_date']

    # Download stock data from yfinance
    stock_data = {}
    for ticker in tqdm(ticker_list, desc='Downloading stock data'):
        stock_data[ticker] = yf.download(ticker, start=start_date, end=end_date, interval=interval, progress=False)

    # Convert the dictionary to a pandas DataFrame with a MultiIndex
    dataset = pd.concat(stock_data, axis=1)

    # re adjust the table only to show the required column (adj. closing price)
    dataset = dataset[dataset.columns[4::6]]
    dataset.columns = dataset.columns.droplevel(1)

    # return raw dataset
    return dataset

In [3]:
config_data = util.load_config()
raw_dataset_ori =read_raw_data(config_data)


1 Failed download:
- TRIL.JK: No timezone found, symbol may be delisted



1 Failed download:
- HDTX.JK: No timezone found, symbol may be delisted



1 Failed download:
- NIPS.JK: No timezone found, symbol may be delisted



1 Failed download:
- SUGI.JK: No timezone found, symbol may be delisted



1 Failed download:
- TRIO.JK: No timezone found, symbol may be delisted


In [4]:
# Keep a copy data to be reuse in case needed later to prevent redownload
raw_dataset = raw_dataset_ori.copy()
raw_dataset

,PACK.JK,VAST.JK,CHIP.JK,HALO.JK,KING.JK,PGEO.JK,FUTR.JK,HILL.JK,BDKR.JK,PTMP.JK,...,CBPE.JK,SUNI.JK,CBRE.JK,WINE.JK,BMBL.JK,PEVE.JK,LAJU.JK,FWCT.JK,NAYZ.JK,IRSX.JK
Date,,,,,,,,,,,,,,,,,,,,,
2023-02-08,146.0,101.0,176.0,117.0,NaN,NaN,NaN,NaN,NaN,NaN,...,148.0,296.0,90.0,372.0,68.0,204.0,173.0,112.0,73.0,101.0
2023-02-09,132.0,96.0,193.0,118.0,NaN,NaN,NaN,NaN,NaN,NaN,...,149.0,298.0,84.0,368.0,64.0,200.0,206.0,128.0,75.0,100.0
2023-02-10,124.0,120.0,212.0,131.0,NaN,NaN,NaN,NaN,NaN,NaN,...,147.0,306.0,88.0,344.0,65.0,200.0,244.0,130.0,72.0,93.0
2023-02-13,112.0,112.0,232.0,122.0,NaN,NaN,NaN,NaN,NaN,NaN,...,152.0,300.0,85.0,324.0,65.0,199.0,228.0,144.0,76.0,94.0
2023-02-14,102.0,108.0,230.0,119.0,NaN,NaN,NaN,NaN,NaN,NaN,...,149.0,312.0,80.0,304.0,65.0,208.0,214.0,134.0,81.0,95.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-03-23,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-05-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-05-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
# change the index format from object into datetime 
raw_dataset.index = pd.to_datetime(raw_dataset.index)

# sort the date index
raw_dataset = raw_dataset.sort_index(ascending=True)

#raw_dataset = raw_dataset.reset_index()

# Delete unrequired rows & columns where all its value is NaN
raw_dataset.dropna(axis=0, how='all', inplace=True)
raw_dataset.dropna(axis=1, how='all', inplace=True)

# Replace NaN value with 0
raw_dataset.fillna(0, inplace=True)

# set date as the index of the dataset
raw_dataset

,PACK.JK,VAST.JK,CHIP.JK,HALO.JK,KING.JK,PGEO.JK,FUTR.JK,HILL.JK,BDKR.JK,PTMP.JK,...,CBPE.JK,SUNI.JK,CBRE.JK,WINE.JK,BMBL.JK,PEVE.JK,LAJU.JK,FWCT.JK,NAYZ.JK,IRSX.JK
Date,,,,,,,,,,,,,,,,,,,,,
2022-01-03,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2022-01-04,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2022-01-05,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2022-01-06,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2022-01-07,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-03-31,155.0,112.0,1460.0,212.0,252.0,695.0,109.0,2020.0,256.0,185.0,...,150.0,292.0,55.0,550.0,72.0,161.0,326.0,98.0,96.0,119.0
2023-04-03,154.0,114.0,1460.0,220.0,264.0,655.0,121.0,2030.0,258.0,190.0,...,149.0,294.0,54.0,585.0,71.0,157.0,318.0,95.0,100.0,140.0
2023-04-04,164.0,114.0,1460.0,260.0,256.0,660.0,115.0,1990.0,258.0,196.0,...,149.0,296.0,56.0,565.0,64.0,182.0,312.0,99.0,105.0,140.0


In [6]:
# Save raw dataset to file
util.pickle_dump(raw_dataset, config_data['raw_dataset_path'])

## 2. Data Definition

## 3. Data Validation

### 3.1 Data type

In [7]:
# Chek data type each variable
raw_dataset.dtypes

PACK.JK    float64
VAST.JK    float64
CHIP.JK    float64
HALO.JK    float64
KING.JK    float64
            ...   
PEVE.JK    float64
LAJU.JK    float64
FWCT.JK    float64
NAYZ.JK    float64
IRSX.JK    float64
Length: 848, dtype: object

In [8]:
# since there are a lot of columns, need to check the dtypes in summarry
# showing that there are only 2 dtypes, datetime & float.
raw_dataset.dtypes.value_counts()

float64    848
dtype: int64

### 3.2 Data Range

In [9]:
raw_dataset.describe()

,PACK.JK,VAST.JK,CHIP.JK,HALO.JK,KING.JK,PGEO.JK,FUTR.JK,HILL.JK,BDKR.JK,PTMP.JK,...,CBPE.JK,SUNI.JK,CBRE.JK,WINE.JK,BMBL.JK,PEVE.JK,LAJU.JK,FWCT.JK,NAYZ.JK,IRSX.JK
count,326.000000,326.000000,326.000000,326.000000,326.000000,326.000000,326.000000,326.000000,326.000000,326.000000,...,326.000000,326.000000,326.000000,326.000000,326.000000,326.000000,326.000000,326.000000,326.000000,326.000000
mean,15.291411,12.696319,95.503067,18.843558,17.978528,67.546012,10.631902,141.779141,17.871166,10.720859,...,28.763804,55.699387,15.911043,62.926380,13.533742,29.490798,40.042945,15.656442,11.592025,13.527607
std,41.797362,34.153992,300.511623,53.915462,55.064126,221.789483,35.710883,496.925836,65.012723,40.572730,...,59.527390,116.310229,37.096150,138.839057,30.702299,69.158135,101.673768,39.616916,30.409044,36.576295
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,166.000000,120.000000,1460.000000,300.000000,264.000000,875.000000,163.000000,2140.000000,268.000000,212.000000,...,192.000000,320.000000,195.000000,705.000000,170.000000,244.000000,398.000000,159.000000,113.000000,170.000000


In [10]:
# Check data statistics, since the column qty is a lot, then we summarize the describe feature in following function
def raw_dataset_describe(dataset):

    # Get the date index statistic info
    date_df = pd.Series(dataset.index).describe(include='datetime64', datetime_is_numeric=True)

    df = dataset.describe()
    # Get the minimum value for each row across all columns and convert it to a DataFrame
    row_min_df = df.min(axis=1).to_frame('Min')

    # Get the maximum value for each row across all columns and convert it to a DataFrame
    row_max_df = df.max(axis=1).to_frame('Max')

    # Concatenate the min and max DataFrames horizontally
    result_df = pd.concat([date_df, row_min_df, row_max_df], axis=1)


    return result_df

In [11]:
# Call Dataset statistics
raw_dataset_describe(raw_dataset)

,Date,Min,Max
count,326,326.000000,326.000000
mean,2022-08-21 01:10:40.490797568,-2760.737532,37031.595092
min,2022-01-03 00:00:00,-16000.001953,0.000000
25%,2022-04-28 06:00:00,-16000.001953,36775.000000
50%,2022-08-22 12:00:00,-16000.001953,38000.000000
75%,2022-12-13 18:00:00,0.000000,39975.000000
max,2023-04-06 00:00:00,46.000000,167198.484375
std,NaN,8.391032,20151.377044


### 3.3 Data Dimension

In [12]:
raw_dataset.shape

(326, 848)

## 4. Data Defense

In [13]:
def check_data(input_data, params, print_errors=True):

    error_messages = []
    error_stock_tickers = []
    #input_data.fillna(0, inplace=True)
    try:
        # Check index data types
        assert input_data.index.dtype == params['datetime_index'], 'an error occurs in index format, should be datetime.'

        # Check index data type & range
        for column in input_data.columns[1:]:
            if input_data[column].dtype != 'float64':
                error_messages.append(f"Column ({column}) has a non-float data type")
                error_stock_tickers.append(column)

            if not (input_data[column] >= 0).sum() == len(input_data):
                error_messages.append(f'an error occurs in {column} column')
                if column not in error_stock_tickers:
                    error_stock_tickers.append(column)
        
        if error_messages:
            total_errors = len(error_messages)
            error_summary = f"\nTotal errors: {total_errors} errors out of {len(input_data.columns)}\n"
            raise AssertionError(error_summary + "\n".join(error_messages))
    
    except AssertionError as e:
        if print_errors:
            print(e)
    
    return error_stock_tickers



In [14]:
# Check the error stock in the dataset
check_data(raw_dataset, config_data)


Total errors: 1 errors out of 848
an error occurs in SCPI.JK column


['SCPI.JK']

In [15]:
# Found error in stock above (SCPI.JK), which after checking through news and yfinance data,
# it is already delisted since 2013. So this stock should be removed since it is considered
# as an anomaly.
error_stock_tickers = check_data(raw_dataset, config_data, print_errors=False)
raw_dataset.drop(error_stock_tickers, axis=1, inplace=True)

In [16]:
# Recheck the data and found no issue
check_data(raw_dataset, config_data)

[]

## 5. Data Splitting

In [17]:
# Initialize TimeSeriesSplit object
n_splits = 3
tscv = TimeSeriesSplit(n_splits = n_splits)

# Get the train & test_val indices at the last split
for train_index, test_val_index in tscv.split(raw_dataset):
    pass

# Calculate the size of the test and validation sets
test_val_size = len(test_val_index)
test_size = val_size = test_val_size // 2

# Define test and validation indices
test_index = test_val_index[:test_size]
val_index = test_val_index[test_size:]

# Extract the train, test, and validation sets
train = raw_dataset.iloc[train_index]
test = raw_dataset.iloc[test_index]
val = raw_dataset.iloc[val_index]




In [18]:
# Split feature and target columns for train, test, and validation sets
feature_columns = raw_dataset.drop([config_data['target']], axis=1).columns
target_column = config_data['target']
X_train, y_train = train[feature_columns], train[target_column]
X_test, y_test = test[feature_columns], test[target_column]
X_val, y_val = val[feature_columns], val[target_column]

print("TRAIN Set:")
display(X_train, display_id='X_train')
display(y_train, display_id='y_train')

print("TEST Set:")
display(X_test, display_id='X_test')
display(y_test, display_id='y_test')

print("VALIDATION Set:")
display(X_val, display_id='X_val')
display(y_val, display_id='y_val')





TRAIN Set:


,PACK.JK,VAST.JK,CHIP.JK,HALO.JK,KING.JK,PGEO.JK,FUTR.JK,HILL.JK,BDKR.JK,PTMP.JK,...,CBPE.JK,SUNI.JK,CBRE.JK,WINE.JK,BMBL.JK,PEVE.JK,LAJU.JK,FWCT.JK,NAYZ.JK,IRSX.JK
Date,,,,,,,,,,,,,,,,,,,,,
2022-01-03,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2022-01-04,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2022-01-05,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2022-01-06,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2022-01-07,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-12-08,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2022-12-09,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2022-12-12,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Date
2022-01-03    3201.196533
2022-01-04    3257.955322
2022-01-05    3189.844727
2022-01-06    3189.844727
2022-01-07    3201.196533
                 ...     
2022-12-08    4780.970703
2022-12-09    4721.505859
2022-12-12    4733.398926
2022-12-13    4709.613281
2022-12-14    4733.398926
Name: BMRI.JK, Length: 245, dtype: float64

TEST Set:


,PACK.JK,VAST.JK,CHIP.JK,HALO.JK,KING.JK,PGEO.JK,FUTR.JK,HILL.JK,BDKR.JK,PTMP.JK,...,CBPE.JK,SUNI.JK,CBRE.JK,WINE.JK,BMBL.JK,PEVE.JK,LAJU.JK,FWCT.JK,NAYZ.JK,IRSX.JK
Date,,,,,,,,,,,,,,,,,,,,,
2022-12-15,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2022-12-16,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2022-12-19,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2022-12-20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2022-12-21,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2022-12-22,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2022-12-23,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2022-12-26,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2022-12-27,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Date
2022-12-15    4709.613281
2022-12-16    4804.756836
2022-12-19    4757.185059
2022-12-20    4780.970703
2022-12-21    4757.185059
2022-12-22    4733.398926
2022-12-23    4721.505859
2022-12-26    4780.970703
2022-12-27    4733.398926
2022-12-28    4745.291992
2022-12-29    4745.291992
2022-12-30    4721.505859
2023-01-02    4697.720215
2023-01-03    4733.398926
2023-01-04    4769.078125
2023-01-05    4673.934082
2023-01-06    4662.041504
2023-01-09    4626.362305
2023-01-10    4412.289062
2023-01-11    4257.680664
2023-01-12    4376.610352
2023-01-13    4364.717285
2023-01-16    4424.182129
2023-01-17    4638.255371
2023-01-18    4543.111816
2023-01-19    4638.255371
2023-01-20    4745.291992
2023-01-23       0.000000
2023-01-24    4709.613281
2023-01-25    4614.469238
2023-01-26    4673.934082
2023-01-27    4769.078125
2023-01-30    4733.398926
2023-01-31    4733.398926
2023-02-01    4614.469238
2023-02-02    4626.362305
2023-02-03    4721.505859
2023-02-06    4697.720215
2023-02

VALIDATION Set:


,PACK.JK,VAST.JK,CHIP.JK,HALO.JK,KING.JK,PGEO.JK,FUTR.JK,HILL.JK,BDKR.JK,PTMP.JK,...,CBPE.JK,SUNI.JK,CBRE.JK,WINE.JK,BMBL.JK,PEVE.JK,LAJU.JK,FWCT.JK,NAYZ.JK,IRSX.JK
Date,,,,,,,,,,,,,,,,,,,,,
2023-02-09,132.0,96.0,193.0,118.0,0.0,0.0,0.0,0.0,0.0,0.0,...,149.0,298.0,84.0,368.0,64.0,200.0,206.0,128.0,75.0,100.0
2023-02-10,124.0,120.0,212.0,131.0,0.0,0.0,0.0,0.0,0.0,0.0,...,147.0,306.0,88.0,344.0,65.0,200.0,244.0,130.0,72.0,93.0
2023-02-13,112.0,112.0,232.0,122.0,0.0,0.0,0.0,0.0,0.0,0.0,...,152.0,300.0,85.0,324.0,65.0,199.0,228.0,144.0,76.0,94.0
2023-02-14,102.0,108.0,230.0,119.0,0.0,0.0,0.0,0.0,0.0,0.0,...,149.0,312.0,80.0,304.0,65.0,208.0,214.0,134.0,81.0,95.0
2023-02-15,94.0,105.0,238.0,126.0,0.0,0.0,0.0,0.0,0.0,0.0,...,149.0,306.0,75.0,322.0,66.0,206.0,200.0,129.0,85.0,91.0
2023-02-16,96.0,108.0,260.0,122.0,117.0,0.0,0.0,0.0,0.0,0.0,...,147.0,306.0,79.0,316.0,65.0,202.0,186.0,127.0,87.0,92.0
2023-02-17,105.0,101.0,286.0,116.0,106.0,0.0,0.0,0.0,0.0,0.0,...,152.0,302.0,74.0,312.0,63.0,199.0,214.0,120.0,91.0,92.0
2023-02-20,97.0,94.0,314.0,113.0,96.0,0.0,0.0,0.0,0.0,0.0,...,150.0,302.0,69.0,310.0,63.0,202.0,226.0,122.0,87.0,91.0
2023-02-21,92.0,88.0,344.0,112.0,105.0,0.0,0.0,0.0,0.0,0.0,...,150.0,300.0,73.0,326.0,62.0,200.0,216.0,118.0,95.0,92.0


Date
2023-02-09     4876.114746
2023-02-10     4899.900391
2023-02-13     4923.686523
2023-02-14     4935.579590
2023-02-15     4864.221680
2023-02-16     4840.435547
2023-02-17     4899.900391
2023-02-20     4888.007812
2023-02-21     4852.328613
2023-02-22     4757.185059
2023-02-23     4792.863770
2023-02-24     4816.649902
2023-02-27     4828.542969
2023-02-28     4757.185059
2023-03-01     4804.756836
2023-03-02     4864.221680
2023-03-03     4780.970703
2023-03-06     4816.649902
2023-03-07     4840.435547
2023-03-08     4888.007812
2023-03-09     4971.258301
2023-03-10     4935.579590
2023-03-13     4923.686523
2023-03-14     4721.505859
2023-03-15     4780.970703
2023-03-16     4685.827148
2023-03-17     4804.756836
2023-03-20     4757.185059
2023-03-21     4995.044434
2023-03-22        0.000000
2023-03-23        0.000000
2023-03-24     5185.331543
2023-03-27     5100.000000
2023-03-28     5062.500000
2023-03-29     5175.000000
2023-03-30    10225.000000
2023-03-31     5162.500

<DisplayHandle display_id=y_val>

In [19]:
util.pickle_dump(X_train, config_data["train_set_path"][0])
util.pickle_dump(y_train, config_data["train_set_path"][1])

util.pickle_dump(X_val, config_data["valid_set_path"][0])
util.pickle_dump(y_val, config_data["valid_set_path"][1])

util.pickle_dump(X_test, config_data["test_set_path"][0])
util.pickle_dump(y_test, config_data["test_set_path"][1])